In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import os
import pandas as pd
import datetime
import numpy as np
import holidays
from scipy import stats

In [46]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [123]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [193]:
df

,Data,Klient - Nazwa g?ówna,Numer oddzia?u,Produkt,Miejsce za?adunku,dytsans,Delivered Qty,Jednostka miary
0,3/1/2019,LmRxYuMmRLINurzReyzrz,16770434,Produkt 21,Punkt2,134.7338667,15.96,5
1,4/1/2019,kmMyiUiuRYkZgr,16883721,Produkt 25,Punkt5,0,16.80,1
2,4/1/2019,SxLMzxuKRzKgr,16770726,Produkt 25,Punkt5,0,6.53,1
3,3/1/2019,SxLMzxuKRzKgr,16770726,Produkt 25,Punkt5,0,5.60,1
4,4/1/2019,KxRyiUiuSxRPxRKgr,16770412,Produkt 27,Punkt4,0,1.87,1
...,...,...,...,...,...,...,...,...
46615,12/29/2020,ixLyzNNuGNImZNx,16917249,Produkt 27,Punkt2,0,15.87,1
46616,12/29/2020,ixLyzNNuGNImZNx,16917249,Produkt 15,Punkt2,0,30.80,1
46617,12/29/2020,ixLyzNNuGNImZNx,16917249,Produkt 27,Punkt2,0,15.87,1
46618,12/29/2020,kxRPINmXuINxrRxk?zr,16819577,Produkt 27,Punkt2,0,6.53,1


In [194]:
df.drop(['Klient - Nazwa g?ówna', 'Jednostka miary'], axis=1, inplace=True)

In [195]:
df.rename(columns={'Numer oddzia?u': 'Numer_oddzialu', 'Miejsce za?adunku': 'Miejsce_zaladunku', 'dytsans': 'dystans'}, inplace=True)

In [196]:
df['Data'] = df['Data'].replace('-', np.nan)
df['Data'] = pd.to_datetime(df['Data'], format='mixed', dayfirst=True, errors='coerce') # Use errors='coerce' to turn any unparseable dates into NaT (Not a Time)

In [197]:
df

,Data,Numer_oddzialu,Produkt,Miejsce_zaladunku,dystans,Delivered Qty
0,2019-01-03,16770434,Produkt 21,Punkt2,134.7338667,15.96
1,2019-01-04,16883721,Produkt 25,Punkt5,0,16.80
2,2019-01-04,16770726,Produkt 25,Punkt5,0,6.53
3,2019-01-03,16770726,Produkt 25,Punkt5,0,5.60
4,2019-01-04,16770412,Produkt 27,Punkt4,0,1.87
...,...,...,...,...,...,...
46615,2020-12-29,16917249,Produkt 27,Punkt2,0,15.87
46616,2020-12-29,16917249,Produkt 15,Punkt2,0,30.80
46617,2020-12-29,16917249,Produkt 27,Punkt2,0,15.87
46618,2020-12-29,16819577,Produkt 27,Punkt2,0,6.53


In [198]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46620 entries, 0 to 46619
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Data               46597 non-null  datetime64[ns]
 1   Numer_oddzialu     46620 non-null  int64         
 2   Produkt            46620 non-null  object        
 3   Miejsce_zaladunku  46620 non-null  object        
 4   dystans            46620 non-null  object        
 5   Delivered Qty      46620 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 2.1+ MB
None


In [199]:
print(df.isnull().sum())

Data                 23
Numer_oddzialu        0
Produkt               0
Miejsce_zaladunku     0
dystans               0
Delivered Qty         0
dtype: int64


In [200]:
df.dropna(subset=['Data'], inplace=True)

In [202]:
print("Unikalne wartości w kolumnie 'Produkt':", df['Produkt'].nunique())
print("Unikalne wartości w kolumnie 'Miejsce_zaladunku':", df['Miejsce_zaladunku'].nunique())
print("Unikalne wartości w kolumnie 'Numer_oddzialu':", df['Numer_oddzialu'].nunique())

Unikalne wartości w kolumnie 'Produkt': 13
Unikalne wartości w kolumnie 'Miejsce_zaladunku': 7
Unikalne wartości w kolumnie 'Numer_oddzialu': 2158


In [203]:
df = df.drop(columns=['Numer_oddzialu'])

In [204]:
df['dystans'] = pd.to_numeric(df['dystans'], errors='coerce')

In [205]:
df = df.dropna(subset=['dystans'])

In [206]:
df

,Data,Produkt,Miejsce_zaladunku,dystans,Delivered Qty
0,2019-01-03,Produkt 21,Punkt2,134.733867,15.96
1,2019-01-04,Produkt 25,Punkt5,0.000000,16.80
2,2019-01-04,Produkt 25,Punkt5,0.000000,6.53
3,2019-01-03,Produkt 25,Punkt5,0.000000,5.60
4,2019-01-04,Produkt 27,Punkt4,0.000000,1.87
...,...,...,...,...,...
46615,2020-12-29,Produkt 27,Punkt2,0.000000,15.87
46616,2020-12-29,Produkt 15,Punkt2,0.000000,30.80
46617,2020-12-29,Produkt 27,Punkt2,0.000000,15.87
46618,2020-12-29,Produkt 27,Punkt2,0.000000,6.53


In [207]:
pl_holidays = holidays.Poland()

In [208]:
df['Data'] = pd.to_datetime(df['Data'])
df['Rok_Tydzien'] = df['Data'].dt.strftime('%Y-%U')
df['Miesiac'] = df['Data'].dt.month
df['TydzienRoku'] = df['Data'].dt.isocalendar().week
pl_holidays = holidays.Poland()
df['is_weekend'] = df['Data'].dt.weekday >= 5
df['is_workday'] = df['Data'].dt.weekday < 5
df['is_holiday'] = df['Data'].isin(pl_holidays)

In [212]:
df['is_weekend_or_holiday'] = df['is_weekend'] | df['is_holiday']

In [213]:
produkty = df['Produkt'].unique()

for produkt in produkty:
    df_temp = df[df['Produkt'] == produkt]

    workday_qty = df_temp[df_temp['is_weekend_or_holiday'] == False]['Delivered Qty']
    weekend_holiday_qty = df_temp[df_temp['is_weekend_or_holiday'] == True]['Delivered Qty']

    if len(workday_qty) < 2 or len(weekend_holiday_qty) < 2:
        print(f"{produkt}: Za mało danych do testu statystycznego")
        continue

    t_stat, p_value = stats.ttest_ind(workday_qty, weekend_holiday_qty, equal_var=False)

    print(f"{produkt}:")
    print(f"  Średnia w dni robocze: {workday_qty.mean():.2f}")
    print(f"  Średnia w weekendy/święta: {weekend_holiday_qty.mean():.2f}")
    print(f"  Test t: t={t_stat:.3f}, p={p_value:.5f}")
    if p_value < 0.05:
        print("  -> Różnica istotna statystycznie (p < 0.05)\n")
    else:
        print("  -> Brak istotnej różnicy (p >= 0.05)\n")

Produkt 21:
  Średnia w dni robocze: 10.97
  Średnia w weekendy/święta: 10.73
  Test t: t=0.720, p=0.47173
  -> Brak istotnej różnicy (p >= 0.05)

Produkt 25:
  Średnia w dni robocze: 15.03
  Średnia w weekendy/święta: 15.13
  Test t: t=-0.381, p=0.70294
  -> Brak istotnej różnicy (p >= 0.05)

Produkt 27:
  Średnia w dni robocze: 12.31
  Średnia w weekendy/święta: 12.41
  Test t: t=-0.933, p=0.35092
  -> Brak istotnej różnicy (p >= 0.05)

Produkt 15:
  Średnia w dni robocze: 13.16
  Średnia w weekendy/święta: 13.19
  Test t: t=-0.266, p=0.79031
  -> Brak istotnej różnicy (p >= 0.05)

Produkt 26:
  Średnia w dni robocze: 10.97
  Średnia w weekendy/święta: 10.91
  Test t: t=0.155, p=0.87720
  -> Brak istotnej różnicy (p >= 0.05)

Produkt 24:
  Średnia w dni robocze: 12.31
  Średnia w weekendy/święta: 12.78
  Test t: t=-0.899, p=0.36908
  -> Brak istotnej różnicy (p >= 0.05)

Produkt 18:
  Średnia w dni robocze: 12.97
  Średnia w weekendy/święta: 13.01
  Test t: t=-0.181, p=0.85666
  -> B

In [214]:
produkty = df['Produkt'].unique()

for produkt in produkty:
    df_temp = df[df['Produkt'] == produkt]


    groups = [group['Delivered Qty'].values for name, group in df_temp.groupby('Miesiac') if len(group) > 1]
    months = [name for name, group in df_temp.groupby('Miesiac') if len(group) > 1]

    if len(groups) < 2:
        print(f"{produkt}: Za mało grup (miesięcy) do testu ANOVA")
        continue

    # Test ANOVA
    f_stat, p_value = stats.f_oneway(*groups)

    print(f"{produkt}:")
    print(f"  Test ANOVA: F={f_stat:.3f}, p={p_value:.5f}")
    if p_value < 0.05:
        print("  -> Istotna różnica między miesiącami (p < 0.05)\n")
    else:
        print("  -> Brak istotnej różnicy między miesiącami (p >= 0.05)\n")

Produkt 21:
  Test ANOVA: F=1.362, p=0.18528
  -> Brak istotnej różnicy między miesiącami (p >= 0.05)

Produkt 25:
  Test ANOVA: F=6.727, p=0.00000
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 27:
  Test ANOVA: F=17.685, p=0.00000
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 15:
  Test ANOVA: F=24.815, p=0.00000
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 26:
  Test ANOVA: F=6.218, p=0.00000
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 24:
  Test ANOVA: F=2.087, p=0.01912
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 18:
  Test ANOVA: F=9.107, p=0.00000
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 23:
  Test ANOVA: F=6.305, p=0.00011
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 1:
  Test ANOVA: F=1.760, p=0.05638
  -> Brak istotnej różnicy między miesiącami (p >= 0.05)

Promocja:
  Test ANOVA: F=5.432, p=0.00000
  -> Istotna różnica między miesiącami (p < 0.05)

Produkt 11:
  Test ANOVA: F

In [209]:
def get_season(month):
    if month in [3,4,5]:
        return 'Wiosna'
    elif month in [6,7,8]:
        return 'Lato'
    elif month in [9,10,11]:
        return 'Jesien'
    else:
        return 'Zima'

df['Pora_roku'] = df['Miesiac'].apply(get_season)

produkty = df['Produkt'].unique()

for produkt in produkty:
    df_temp = df[df['Produkt'] == produkt]

    groups = [group['Delivered Qty'].values for name, group in df_temp.groupby('Pora_roku') if len(group) > 1]
    seasons = [name for name, group in df_temp.groupby('Pora_roku') if len(group) > 1]

    if len(groups) < 2:
        print(f"{produkt}: Za mało grup do testu ANOVA")
        continue

    f_stat, p_value = stats.f_oneway(*groups)

    print(f"{produkt}:")
    print(f"  Test ANOVA (pory roku): F={f_stat:.3f}, p={p_value:.5f}")
    if p_value < 0.05:
        print("  -> Istotna różnica między porami roku (p < 0.05)\n")
    else:
        print("  -> Brak istotnej różnicy między porami roku (p >= 0.05)\n")

Produkt 21:
  Test ANOVA (pory roku): F=3.148, p=0.02438
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 25:
  Test ANOVA (pory roku): F=9.252, p=0.00000
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 27:
  Test ANOVA (pory roku): F=34.485, p=0.00000
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 15:
  Test ANOVA (pory roku): F=68.960, p=0.00000
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 26:
  Test ANOVA (pory roku): F=12.969, p=0.00000
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 24:
  Test ANOVA (pory roku): F=0.812, p=0.48748
  -> Brak istotnej różnicy między porami roku (p >= 0.05)

Produkt 18:
  Test ANOVA (pory roku): F=14.183, p=0.00000
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 23:
  Test ANOVA (pory roku): F=8.090, p=0.00510
  -> Istotna różnica między porami roku (p < 0.05)

Produkt 1:
  Test ANOVA (pory roku): F=0.925, p=0.42799
  -> Brak istotnej różnicy między porami roku (p >= 0.05)

Pr

In [210]:
# Grupowanie tygodniowe + dodatkowe dane kalendarzowe
df_grouped = df.groupby(['Rok_Tydzien', 'Produkt', 'Miejsce_zaladunku', 'Pora_roku','TydzienRoku']).agg({
    'Delivered Qty': 'sum',
    'dystans': 'mean',
}).reset_index()

# Sortowanie wg Produkt, Miejsce, Rok_Tydzien
df_grouped = df_grouped.sort_values(['Produkt', 'Miejsce_zaladunku', 'Rok_Tydzien'])

# Zmiana Delivered Qty (target)
df_grouped['Delivered_Delta'] = df_grouped.groupby(['Produkt', 'Miejsce_zaladunku'])['Delivered Qty'].diff().fillna(0)

# Tworzenie cech lagowanych (1-3 tygodnie wstecz) dla targetu i dystans.
for lag in [1, 2, 3]:
    df_grouped[f'Delivered_Delta_lag{lag}'] = df_grouped.groupby(['Produkt', 'Miejsce_zaladunku'])['Delivered_Delta'].shift(lag)
    df_grouped[f'dystans_lag{lag}'] = df_grouped.groupby(['Produkt', 'Miejsce_zaladunku'])['dystans'].shift(lag)

# Usuwanie wierszy z brakami po lagach
df_grouped = df_grouped.dropna()

# Kodowanie kategorii (Produkt, Miejsce_zaladunku, Pora_roku)
df_grouped = pd.get_dummies(df_grouped, columns=['Produkt', 'Miejsce_zaladunku', 'Pora_roku'], drop_first=True)

# Przygotowanie cech i targetu
exclude_cols = ['Rok_Tydzien', 'Delivered Qty', 'Delivered_Delta']
feature_cols = [col for col in df_grouped.columns if col not in exclude_cols]

X = df_grouped[feature_cols]
y = df_grouped['Delivered_Delta']

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

r2_list = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = XGBRegressor(objective='reg:squarederror', n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    r2 = r2_score(y_val, preds)

    print(f"Fold {fold+1}:  R2 = {r2:.3f}")
    r2_list.append(r2)

print(f"Average R2: {sum(r2_list)/len(r2_list):.3f}")

Fold 1:  R2 = 0.661
Fold 2:  R2 = 0.703
Fold 3:  R2 = 0.428
Fold 4:  R2 = 0.846
Fold 5:  R2 = 0.690
Average R2: 0.666
